In [1]:
# Packages, seed and path
## packages
from edge_sim_py import *
import math
import os
import random
import msgpack
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from DQN import DQNAgent

## seed
import torch
torch.manual_seed(5)
import random
random.seed(5)
import numpy as np
np.random.seed(5)

## path
algo_name = "qlearning_lm"

/home/mascalmeida/miniconda3/envs/edgeai/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Functions
# def custom_collect_method(self) -> dict: # Custom collect method to measure the power consumption of each server
#     metrics = {
#         "Instance ID": self.id,
#         "Power Consumption": self.get_power_consumption(),
#     }
#     return metrics

global agent
global reward_list
global reward_count_list
global reward_count

reward_list = list()
power_list = list() # List to store total power consumption everytime the task scheduling algorithm is used


def my_algorithm(parameters):
  
    print("\n\n")
    total_reward = 0
    total_power = 0 #We sum the power consumption after migrating each service

    for service in Service.all(): #Iterate over every service
        
        #If service needs to be migrated
        if not service.being_provisioned:

            
            #Initialise our state vector, which is the concatenation of the cpu,memory,disk utilisation and current power consumption

            state_vector = []

            for edge_server in EdgeServer.all():
                edge_server_cpu = edge_server.cpu
                edge_server_memory = edge_server.memory
                edge_server_disk = edge_server.disk
                power = (edge_server_cpu * edge_server_memory * edge_server_disk) ** (1 / 3)
                vector = [edge_server_cpu, edge_server_memory, edge_server_disk, power]
                state_vector = state_vector + vector


            #Pass the state vector to out Q - learning agent, and retrieve action
            state_vector = np.array(state_vector)    
            action = agent.choose_action(state_vector)

            
            #To conserve resources, we don't want to migrate back to our host 
            if EdgeServer.all()[action] == service.server:
                break

            print(f"[STEP {parameters['current_step']}] Migrating {service} From {service.server} to {EdgeServer.all()[action]}")

            
            #Migrate service to new edgeserver
            service.provision(target_server=EdgeServer.all()[action])

            
            #Get our next state, after taking action
            next_state_vector = []
            reward = 0
            power = 0

            for edge_server in EdgeServer.all():
                edge_server_cpu = edge_server.cpu
                edge_server_memory = edge_server.memory
                edge_server_disk = edge_server.disk
                power = (edge_server_cpu * edge_server_memory * edge_server_disk) ** (1 / 3)
                vector = [edge_server_cpu, edge_server_memory, edge_server_disk, power]
                next_state_vector = next_state_vector + vector
                reward = reward + 1/edge_server.get_power_consumption() #Our reward is the inverse of the edge server's power consumption
                power = power + edge_server.get_power_consumption() #get the sum of powerconsumption of each edge server 



            #Pass the updated state and reward for backporpogation through Q - network
            next_state_vector = np.array(next_state_vector)
            agent.update(state_vector,action,next_state_vector,reward,False)

            #print(reward)
            total_reward += reward
            total_power += power #Sum our power consumption
    

    reward_list.append(total_reward)
    power_list.append(total_power) #Append power consumption to power list for plotting
    agent.epsilon*=agent.epsilon_decay #Reduce the probability of agent taking random action for exploration

def stopping_criterion(model: object):    
    # As EdgeSimPy will halt the simulation whenever this function returns True,
    # its output will be a boolean expression that checks if the current time step is 600
    return model.schedule.steps == 1000

In [3]:
simulator = Simulator(
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
#simulator.initialize(input_file="sample_dataset3.json")
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

#Assigning the custom collect method
#EdgeServer.collect = custom_collect_method

#Initialise of DQN agent with state and action dimension
#Here, state is the current cpu, memory and disk utilisation of the server, and action space is the choice of edge server
#i.e. the Edge server with the maximum Q- value will be migrated to
agent = DQNAgent(len(EdgeServer.all()) * 4, len(EdgeServer.all()))

# Executing the simulation
simulator.run_model()





[STEP 1] Migrating Service_1 From None to EdgeServer_6
[STEP 1] Migrating Service_2 From None to EdgeServer_1
[STEP 1] Migrating Service_3 From None to EdgeServer_5
[STEP 1] Migrating Service_4 From None to EdgeServer_4
[STEP 1] Migrating Service_5 From None to EdgeServer_1
[STEP 1] Migrating Service_6 From None to EdgeServer_2



[STEP 2] Migrating Service_3 From EdgeServer_5 to EdgeServer_1






[STEP 4] Migrating Service_1 From EdgeServer_6 to EdgeServer_5
[STEP 4] Migrating Service_2 From EdgeServer_1 to EdgeServer_3



[STEP 5] Migrating Service_1 From EdgeServer_5 to EdgeServer_2



[STEP 6] Migrating Service_2 From EdgeServer_3 to EdgeServer_2



[STEP 7] Migrating Service_1 From EdgeServer_2 to EdgeServer_3
[STEP 7] Migrating Service_2 From EdgeServer_2 to EdgeServer_6



[STEP 8] Migrating Service_1 From EdgeServer_3 to EdgeServer_1
[STEP 8] Migrating Service_2 From EdgeServer_6 to EdgeServer_5
[STEP 8] Migrating Service_5 From EdgeServer_1 to EdgeServer_4



[STEP 9] Migr

In [4]:
# Results
## Retrieving logs dataframe for plot
logs_containerregistry = pd.DataFrame(simulator.agent_metrics["ContainerRegistry"])
logs_edgeserver = pd.DataFrame(simulator.agent_metrics["EdgeServer"])
logs_networkflow = pd.DataFrame(simulator.agent_metrics["NetworkFlow"])
logs_networkswitch = pd.DataFrame(simulator.agent_metrics["NetworkSwitch"])
logs_service = pd.DataFrame(simulator.agent_metrics["Service"])
logs_user = pd.DataFrame(simulator.agent_metrics["User"])

In [5]:
logs_edgeserver['CPU Usage'] = (logs_edgeserver['CPU Demand']*100)/logs_edgeserver['CPU']
logs_edgeserver['RAM Usage'] = (logs_edgeserver['RAM Demand']*100)/logs_edgeserver['RAM']
logs_edgeserver[['Object', 'Power Consumption', 'CPU Usage', 'RAM Usage']].groupby(by=['Object']).mean()

,Power Consumption,CPU Usage,RAM Usage
Object,,,
EdgeServer_1,221.691932,56.256244,56.256244
EdgeServer_2,169.556583,3.596404,3.596404
EdgeServer_3,71.460561,2.959540,5.919081
EdgeServer_4,73.138853,4.070929,8.141858
EdgeServer_5,77.619697,10.606061,9.659091
EdgeServer_6,67.761347,3.404928,5.107393
